In [1]:
import os
import sys

sys.path.append(os.path.abspath(".."))

In [2]:
from typing import Dict, List, Optional

from smolagents import tools
import pandas as pd
import numpy as np

from src.agent.adapters.tools.base import BaseTool

In [3]:
## monkey patching

tools.AUTHORIZED_TYPES = [
    "string",
    "boolean",
    "integer",
    "number",
    "image",
    "audio",
    "array",
    "object",
    "any",
    "null",
    "list",
    "dict",
]

In [4]:
class GetData(BaseTool):
    name = "get_data"
    description = """Get data from an asset."""
    inputs = {
        "asset_ids": {"type": "list", "description": "list of asset ids"},
        "start_date": {"type": "string", "description": "start date", "nullable": True},
        "end_date": {"type": "string", "description": "end date", "nullable": True},
        "aggregation": {
            "type": "string",
            "description": "data aggregation",
            "nullable": True,
            "allowed": ["day", "minute", "hour"],
        },
        "last_value": {
            "type": "boolean",
            "description": "last value",
            "nullable": True,
        },
    }
    outputs = {"data": {"type": "dataframe", "description": "sensor data of an asset"}}
    output_type = "dict"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def forward(
        self,
        asset_ids: List[str],
        start_date: Optional[str] = None,
        end_date: Optional[str] = None,
        aggregation: Optional[str] = "day",
        last_value: bool = False,
    ) -> Dict[str, List[str]]:
        df = pd.DataFrame()

        asset_ids = self.format_input(asset_ids)

        if not last_value:
            aggregation = self.map_aggregation(aggregation.lower())
            start_date = self.convert_to_iso_format(start_date)
            end_date = self.convert_to_iso_format(end_date)

        body = {
            "last_value": last_value,
            "start_date": start_date,
            "end_date": end_date,
            "aggregation": aggregation,
        }

        for asset_id in asset_ids:
            api_url = f"{self.base_url}/v1/data/{asset_id}"

            out = self.call_api(api_url, body=body)

            if out:
                _df = pd.DataFrame.from_dict(out)
                _df.set_index("timestamp", inplace=True)
                _df.drop(["pk_id", "asset_id"], inplace=True, axis=1)
                _df.columns = [asset_id]

                df = pd.merge(df, _df, left_index=True, right_index=True, how="outer")

        df.replace(np.nan, None, inplace=True)
        df.sort_index(inplace=True)

        return {"data": df}

    def map_aggregation(self, aggregation: str) -> str:
        """
        Map the aggregation to the correct aggregation type.
        """
        if aggregation not in ["day", "hour", "minute", "d", "h", "min"]:
            raise ValueError(
                f"Invalid aggregation: {aggregation} - only day, hour, minute, d, h, min are allowed"
            )

        if aggregation == "day" or aggregation == "d":
            aggregation = "d"
        elif aggregation == "hour" or aggregation == "h":
            aggregation = "h"
        elif aggregation == "minute" or aggregation == "min":
            aggregation = "min"
        else:
            aggregation = "d"

        return aggregation

In [5]:
kwargs = {"base_url": "http://localhost:5050"}

In [6]:
tool = GetData(**kwargs)

In [96]:
ids = [
    "a1ff2d3a-6fa9-492a-af51-cfd579cb2308",
    "52f50206-c6b9-47a4-bead-fe791f71cb7c",
    "9b71065a-2273-4173-9785-2487d1573dd7",
]

In [97]:
out = tool.forward(
    ids,
    start_date="2025-04-01T00:00:00",
    end_date="2025-04-01T23:59:00",
    aggregation="min",
)

2025-05-13 10:17:38.703 | INFO     | base:call_api:71 - Fetching data from http://localhost:5050/v1/data/52f50206-c6b9-47a4-bead-fe791f71cb7c with params: {'last_value': False, 'start_date': '2025-04-01T00:00:00', 'end_date': '2025-04-01T23:59:00', 'aggregation': 'min', 'offset': 0, 'limit': 100}
2025-05-13 10:17:39.546 | INFO     | base:call_api:71 - Fetching data from http://localhost:5050/v1/data/52f50206-c6b9-47a4-bead-fe791f71cb7c with params: {'last_value': False, 'start_date': '2025-04-01T00:00:00', 'end_date': '2025-04-01T23:59:00', 'aggregation': 'min', 'offset': 100, 'limit': 100}
2025-05-13 10:17:40.011 | INFO     | base:call_api:71 - Fetching data from http://localhost:5050/v1/data/52f50206-c6b9-47a4-bead-fe791f71cb7c with params: {'last_value': False, 'start_date': '2025-04-01T00:00:00', 'end_date': '2025-04-01T23:59:00', 'aggregation': 'min', 'offset': 200, 'limit': 100}
2025-05-13 10:17:40.472 | INFO     | base:call_api:71 - Fetching data from http://localhost:5050/v1/da

In [100]:
# out["data"].to_csv("temp.csv", index=False)

In [101]:
out = tool.forward(
    ids,
    start_date="2025-04-01T00:00:00",
    end_date="2025-04-01T23:59:00",
    aggregation="h",
)

2025-05-13 10:18:20.793 | INFO     | base:call_api:71 - Fetching data from http://localhost:5050/v1/data/52f50206-c6b9-47a4-bead-fe791f71cb7c with params: {'last_value': False, 'start_date': '2025-04-01T00:00:00', 'end_date': '2025-04-01T23:59:00', 'aggregation': 'h', 'offset': 0, 'limit': 100}
2025-05-13 10:18:20.855 | INFO     | base:call_api:83 - Reached the last page.
2025-05-13 10:18:20.857 | INFO     | base:call_api:71 - Fetching data from http://localhost:5050/v1/data/9b71065a-2273-4173-9785-2487d1573dd7 with params: {'last_value': False, 'start_date': '2025-04-01T00:00:00', 'end_date': '2025-04-01T23:59:00', 'aggregation': 'h', 'offset': 0, 'limit': 100}
2025-05-13 10:18:20.905 | INFO     | base:call_api:83 - Reached the last page.
2025-05-13 10:18:20.905 | INFO     | base:call_api:71 - Fetching data from http://localhost:5050/v1/data/a1ff2d3a-6fa9-492a-af51-cfd579cb2308 with params: {'last_value': False, 'start_date': '2025-04-01T00:00:00', 'end_date': '2025-04-01T23:59:00', '

In [102]:
out["data"]

,52f50206-c6b9-47a4-bead-fe791f71cb7c,a1ff2d3a-6fa9-492a-af51-cfd579cb2308
timestamp,,
2025-04-01T00:00:00,1.0,6.827667
2025-04-01T01:00:00,1.0,6.899667
2025-04-01T02:00:00,1.0,7.311667
2025-04-01T03:00:00,1.0,7.403333
2025-04-01T04:00:00,1.0,7.182833
2025-04-01T05:00:00,1.0,7.019667
2025-04-01T06:00:00,1.0,7.065333
2025-04-01T07:00:00,1.0,6.290333
2025-04-01T08:00:00,1.0,6.831333


In [103]:
out = tool.forward(
    ids,
    start_date="2025-04-01T00:00:00",
    end_date="2025-04-01T23:59:00",
    aggregation="d",
)

2025-05-13 10:18:29.126 | INFO     | base:call_api:71 - Fetching data from http://localhost:5050/v1/data/52f50206-c6b9-47a4-bead-fe791f71cb7c with params: {'last_value': False, 'start_date': '2025-04-01T00:00:00', 'end_date': '2025-04-01T23:59:00', 'aggregation': 'd', 'offset': 0, 'limit': 100}
2025-05-13 10:18:29.175 | INFO     | base:call_api:83 - Reached the last page.
2025-05-13 10:18:29.176 | INFO     | base:call_api:71 - Fetching data from http://localhost:5050/v1/data/9b71065a-2273-4173-9785-2487d1573dd7 with params: {'last_value': False, 'start_date': '2025-04-01T00:00:00', 'end_date': '2025-04-01T23:59:00', 'aggregation': 'd', 'offset': 0, 'limit': 100}
2025-05-13 10:18:29.198 | INFO     | base:call_api:83 - Reached the last page.
2025-05-13 10:18:29.198 | INFO     | base:call_api:71 - Fetching data from http://localhost:5050/v1/data/a1ff2d3a-6fa9-492a-af51-cfd579cb2308 with params: {'last_value': False, 'start_date': '2025-04-01T00:00:00', 'end_date': '2025-04-01T23:59:00', '

In [104]:
out["data"]

,52f50206-c6b9-47a4-bead-fe791f71cb7c,a1ff2d3a-6fa9-492a-af51-cfd579cb2308
timestamp,,
2025-04-01T00:00:00,1.0,7.036118


In [105]:
ids = [
    None,
    "12",
    12,
    "PA101",
    "a1ff2d3a-6fa9-492a-af51-cfd579cb2308",
    "52f50206-c6b9-47a4-bead-fe791f71cb7c",
    "9b71065a-2273-4173-9785-2487d1573dd7",
]

In [106]:
out = tool.forward(ids, last_value=True)

2025-05-13 10:18:34.033 | INFO     | base:call_api:71 - Fetching data from http://localhost:5050/v1/data/a1ff2d3a-6fa9-492a-af51-cfd579cb2308 with params: {'last_value': True, 'start_date': None, 'end_date': None, 'aggregation': 'day', 'offset': 0, 'limit': 100}
2025-05-13 10:18:34.401 | INFO     | base:call_api:83 - Reached the last page.
2025-05-13 10:18:34.405 | INFO     | base:call_api:71 - Fetching data from http://localhost:5050/v1/data/12 with params: {'last_value': True, 'start_date': None, 'end_date': None, 'aggregation': 'day', 'offset': 0, 'limit': 100}
2025-05-13 10:18:34.439 | DEBUG    | base:call_api:90 - HTTP error fetching name for http://localhost:5050/v1/data/12: 422 - {"detail":[{"type":"uuid_parsing","loc":["path","asset_id"],"msg":"Input should be a valid UUID, invalid length: expected length 32 for simple format, found 2","input":"12","ctx":{"error":"invalid length: expected length 32 for simple format, found 2"}}]}
2025-05-13 10:18:34.440 | INFO     | base:call_a

In [107]:
out["data"]

,a1ff2d3a-6fa9-492a-af51-cfd579cb2308,52f50206-c6b9-47a4-bead-fe791f71cb7c
timestamp,,
2025-12-31T23:59:00,0.0,1.0


In [19]:
ids = ["84200101-d718-45bf-bfae-529513586ddd", "18b04353-839d-40a1-84c1-9b547d09dd80"]

In [22]:
out = tool.forward(
    ids[0],
    start_date="2025-04-01T00:00:00",
    end_date="2025-04-01T23:59:00",
    aggregation="h",
)
out2 = tool.forward(
    ids[1],
    start_date="2025-04-01T00:00:00",
    end_date="2025-04-01T23:59:00",
    aggregation="h",
)

2025-05-13 21:31:44.183 | INFO     | src.agent.adapters.tools.base:call_api:74 - Fetching data from http://localhost:5050/v1/data/84200101-d718-45bf-bfae-529513586ddd with params: {'last_value': False, 'start_date': '2025-04-01T00:00:00', 'end_date': '2025-04-01T23:59:00', 'aggregation': 'h', 'offset': 0, 'limit': 100}
2025-05-13 21:31:44.252 | INFO     | src.agent.adapters.tools.base:call_api:88 - Reached the last page.
2025-05-13 21:31:44.254 | INFO     | src.agent.adapters.tools.base:call_api:74 - Fetching data from http://localhost:5050/v1/data/18b04353-839d-40a1-84c1-9b547d09dd80 with params: {'last_value': False, 'start_date': '2025-04-01T00:00:00', 'end_date': '2025-04-01T23:59:00', 'aggregation': 'h', 'offset': 0, 'limit': 100}
2025-05-13 21:31:44.295 | INFO     | src.agent.adapters.tools.base:call_api:88 - Reached the last page.


,84200101-d718-45bf-bfae-529513586ddd,18b04353-839d-40a1-84c1-9b547d09dd80
timestamp,,
2025-04-01T00:00:00,3.049167,1.615167
2025-04-01T01:00:00,1.710667,1.746167
2025-04-01T02:00:00,2.032167,1.511000
2025-04-01T03:00:00,2.664500,1.356500
2025-04-01T04:00:00,2.351500,1.099000
2025-04-01T05:00:00,1.968167,1.725500
2025-04-01T06:00:00,1.796667,1.248333
2025-04-01T07:00:00,1.876000,1.684333
2025-04-01T08:00:00,1.761667,1.817000
